# **Instalando e importando a biblioteca Pandas**

In [107]:
pip install pandas

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.0.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [108]:
import pandas as pd

# **Extraindo os dados**

In [109]:
arquivos = {
    '2021': 'dados/MICRODADOS_ENEM_2021_CORTADO.csv',
    '2022': 'dados/MICRODADOS_ENEM_2022_CORTADO.csv',
    '2023': 'dados/MICRODADOS_ENEM_2023_CORTADO.csv'
}

lista_dataframes = []

for ano, arquivo in arquivos.items():
    try:
        df = pd.read_csv(arquivo, encoding='latin-1', sep=',')
        lista_dataframes.append(df)
    except Exception as e:
        print(f"Erro ao ler os dados do ano {ano}: {e}")

df = pd.concat(lista_dataframes, ignore_index=True)

# **Visualização geral**

In [110]:
display(df.head(5))
display(df.info())

,NU_INSCRICAO,NU_ANO,TP_FAIXA_ETARIA,TP_SEXO,TP_ESTADO_CIVIL,TP_COR_RACA,TP_NACIONALIDADE,TP_ST_CONCLUSAO,TP_ANO_CONCLUIU,TP_ESCOLA,...,Q016,Q017,Q018,Q019,Q020,Q021,Q022,Q023,Q024,Q025
0,210053865474,2021,5,F,1,1,1,1,3,1,...,A,A,A,B,A,A,B,A,B,B
1,210052384164,2021,12,M,1,1,1,1,11,1,...,A,A,A,B,A,A,C,A,A,A
2,210052589243,2021,13,F,3,1,1,1,15,1,...,B,A,A,B,A,A,C,B,B,B
3,210052128335,2021,3,M,1,3,1,2,0,2,...,A,A,A,B,A,A,B,A,B,B
4,210051353021,2021,2,F,1,3,1,2,0,2,...,B,A,A,B,A,B,E,A,B,B


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 600000 entries, 0 to 599999
Data columns (total 76 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   NU_INSCRICAO            600000 non-null  int64  
 1   NU_ANO                  600000 non-null  int64  
 2   TP_FAIXA_ETARIA         600000 non-null  int64  
 3   TP_SEXO                 600000 non-null  object 
 4   TP_ESTADO_CIVIL         600000 non-null  int64  
 5   TP_COR_RACA             600000 non-null  int64  
 6   TP_NACIONALIDADE        600000 non-null  int64  
 7   TP_ST_CONCLUSAO         600000 non-null  int64  
 8   TP_ANO_CONCLUIU         600000 non-null  int64  
 9   TP_ESCOLA               600000 non-null  int64  
 10  TP_ENSINO               238926 non-null  float64
 11  IN_TREINEIRO            600000 non-null  int64  
 12  CO_MUNICIPIO_ESC        152097 non-null  float64
 13  NO_MUNICIPIO_ESC        152097 non-null  object 
 14  CO_UF_ESC           

None

## **Transformação da coluna TB_ENSINO**

In [111]:
print("Valores únicos da coluna 'Ensino': \n", df['TP_ENSINO'].unique())
print("Coluna TP_ENSINO")
print(df['TP_ENSINO'].head(5))

Valores únicos da coluna 'Ensino': 
 [nan  1.  2.]
Coluna TP_ENSINO
0    NaN
1    NaN
2    NaN
3    1.0
4    1.0
Name: TP_ENSINO, dtype: float64


A coluna TB_ENSINO possui 3 valores, NaN, 1 e 2. Os valores NaN serão alterados para 0, para assim ser possível transformar o tipo para inteiro. Depois disso os valores serão alterados segundo a legenda dos microdados: 1 para Ensino Regulas e 2 para Educação Especial - Modalidade Substitutiva, o 0 será tratado como Não informado. O nome da tabela passará a ser ensino.

In [112]:
dicio_ensino = {
    0: 'Não informado',
    1: 'Ensino Regular',
    2: 'Educação Especial - Modalidade Substitutiva'
}

df['TP_ENSINO'] = (df['TP_ENSINO'].fillna(0).astype(int))

df['TP_ENSINO'] = df['TP_ENSINO'].map(dicio_ensino)

print("Coluna TP_ENSINO")
print(df['TP_ENSINO'].head(5))
print("Valores únicos da coluna 'TB_ENSINO': \n", df['TP_ENSINO'].unique())


Coluna TP_ENSINO
0     Não informado
1     Não informado
2     Não informado
3    Ensino Regular
4    Ensino Regular
Name: TP_ENSINO, dtype: object
Valores únicos da coluna 'TB_ENSINO': 
 ['Não informado' 'Ensino Regular'
 'Educação Especial - Modalidade Substitutiva']


In [113]:
df.rename(columns={'TP_ENSINO': 'ensino'}, inplace=True)
print("Coluna ensino")
print(df['ensino'].head(5))

Coluna ensino
0     Não informado
1     Não informado
2     Não informado
3    Ensino Regular
4    Ensino Regular
Name: ensino, dtype: object


## **Transformação da coluna Q006**

In [114]:
print("Valores únicos da coluna 'Q006': \n", df['Q006'].unique())
print('Coluna Q006')
print(df['Q006'].head(5))

Valores únicos da coluna 'Q006': 
 ['D' 'B' 'C' 'G' 'H' 'F' 'E' 'A' 'I' 'L' 'J' 'N' 'O' 'K' 'M' 'Q' 'P']
Coluna Q006
0    D
1    B
2    C
3    B
4    C
Name: Q006, dtype: object


A coluna Q006 possui muitas respostas possíveis, indo do A ao Q, além do NaN. Por isso, agrupamos algumas das respostas possíveis, ficando: 'Nenhuma renda', 'Até R$ 1.320,00', 'De R$ 1.320,01 até R$ 3.400,00', 'De R$ 3.400,01 até R$ 8.100,00', 'De R$ 8.100,01 até R$ 25.200,00', 'Acima de R$ 25.200,01', os valores NaN serão tratados com 'Não informado'. As categorias foram alocadas de acordo com o valor mais próximo da resposta original. Além disso, trocamos o nome da coluna para 'renda_mensal'

In [115]:


df['Q006'] = df['Q006'].fillna('Não informado')

df.loc[df['Q006'] == 'A', 'Q006'] = 'Nenhuma renda'
df.loc[df['Q006'] == 'B', 'Q006'] = 'Até R$ 1.320,00'
df.loc[df['Q006'].isin(['C', 'D', 'E', 'F']), 'Q006'] = 'De R$ 1.320,01 até R$ 3.400,00'
df.loc[df['Q006'].isin(['G', 'H', 'I', 'J']), 'Q006'] = 'De R$ 3.400,01 até R$ 8.100,00'
df.loc[df['Q006'].isin(['K', 'L', 'M', 'N', 'O', 'P']), 'Q006'] = 'De R$ 8.100,01 até R$ 25.200,00'
df.loc[df['Q006'] == 'Q', 'Q006'] = 'Acima de R$ 25.200,01'


print("Coluna Q006")
print(df['Q006'].head(5))
print("Valores únicos da coluna 'Q006': \n", df['Q006'].unique())

Coluna Q006
0    De R$ 1.320,01 até R$ 3.400,00
1                   Até R$ 1.320,00
2    De R$ 1.320,01 até R$ 3.400,00
3                   Até R$ 1.320,00
4    De R$ 1.320,01 até R$ 3.400,00
Name: Q006, dtype: object
Valores únicos da coluna 'Q006': 
 ['De R$ 1.320,01 até R$ 3.400,00' 'Até R$ 1.320,00'
 'De R$ 3.400,01 até R$ 8.100,00' 'Nenhuma renda'
 'De R$ 8.100,01 até R$ 25.200,00' 'Acima de R$ 25.200,01']


In [116]:
df.rename(columns={'Q006': 'renda_mensal'}, inplace=True)
print("Coluna renda_mensal")
print(df['renda_mensal'].head(5))

Coluna renda_mensal
0    De R$ 1.320,01 até R$ 3.400,00
1                   Até R$ 1.320,00
2    De R$ 1.320,01 até R$ 3.400,00
3                   Até R$ 1.320,00
4    De R$ 1.320,01 até R$ 3.400,00
Name: renda_mensal, dtype: object


## **Transformação da coluna Q024**

In [117]:
print("Valores únicos da coluna 'Q024': \n", df['Q024'].unique())
print('Coluna Q024')
print(df['Q024'].head(5))

Valores únicos da coluna 'Q024': 
 ['B' 'A' 'C' 'D' 'E']
Coluna Q024
0    B
1    A
2    B
3    B
4    B
Name: Q024, dtype: object


A coluna Q024 possui 6 valores possíveis, as colunas com NaN foram preenchidas com Não informado, e cada uma das outras respostas foram trocadas pelas informações correspondes no dicionário de microdados, sendo elas: 'A' igual a 'Não.', 'B' igual a 'Sim, um.', 'C' igual a 'Sim, dois.', 'D' igual a 'Sim, três.', 'E' igual a 'Sim, quatro ou mais.', 'F' igual a 'Não informado'. O nome da coluna foi trocado para qtd_computador.

In [118]:
dicio_computador = {
    'A': 'Não.',
    'B': 'Sim, um.',
    'C': 'Sim, dois.',
    'D': 'Sim, três.',
    'E': 'Sim, quatro ou mais.',
    'F': 'Não informado'
}

df['Q024'] = (df['Q024'].fillna('F'))

df['Q024'] = df['Q024'].map(dicio_computador)

print("Coluna Q024")
print(df['Q024'].head(5))
print("Valores únicos da coluna 'Q024': \n", df['Q024'].unique())


Coluna Q024
0    Sim, um.
1        Não.
2    Sim, um.
3    Sim, um.
4    Sim, um.
Name: Q024, dtype: object
Valores únicos da coluna 'Q024': 
 ['Sim, um.' 'Não.' 'Sim, dois.' 'Sim, três.' 'Sim, quatro ou mais.']


In [119]:
df.rename(columns={'Q024': 'qtd_computador'}, inplace=True)
print("Coluna qtd_computador")
print(df['qtd_computador'].head(5))

Coluna qtd_computador
0    Sim, um.
1        Não.
2    Sim, um.
3    Sim, um.
4    Sim, um.
Name: qtd_computador, dtype: object


## **Transformação da coluna TP_FAIXA_ETARIA**

In [120]:
print("Valores únicos da coluna 'TP_FAIXA_ETARIA': \n", df['TP_FAIXA_ETARIA'].unique())
print('Coluna TP_FAIXA_ETARIA')
print(df['TP_FAIXA_ETARIA'].head(20))

Valores únicos da coluna 'TP_FAIXA_ETARIA': 
 [ 5 12 13  3  2 10  9  4  8 11  7  1  6 14 16 19 15 20 17 18]
Coluna TP_FAIXA_ETARIA
0      5
1     12
2     13
3      3
4      2
5     10
6      9
7      4
8      8
9      4
10     5
11     2
12    11
13     5
14     5
15     4
16     7
17     8
18     2
19    11
Name: TP_FAIXA_ETARIA, dtype: int64


TP_FAIXA_ETARIA tem 20 possiveis valores, sendo: (1 = < 17 anos), (de 2 até 10 = 17, 18, 19, ..., 25), (11 = entre 26 e 30), (12 = entre 31 e 35), 
(13 = entre 36 e 40), (14 = entre 41 e 45), (15 = entre 46 e 50), (16 = entre 51 e 55), (17 = entre 56 e 60), (18 = entre 61 e 65), (19 = entre 66 e 70),
(20 = 70+). O tipo de dado será convertido para object.

In [121]:
dicio_faixaetaria = {
    1: 'Menos que 17 anos',
    2: '17 anos',
    3: '18 anos',
    4: '19 anos',
    5: '20 anos',
    6: '21 anos',
    7: '22 anos',
    8: '23 anos',
    9: '24 anos',
    10: '25 anos',
    11: 'entre 26 e 30 anos',
    12: 'entre 31 e 35 anos',
    13: 'entre 36 e 40 anos',
    14: 'entre 41 e 45 anos',
    15: 'entre 46 e 50 anos',
    16: 'entre 51 e 55 anos',
    17: 'entre 56 e 60 anos',
    18: 'entre 61 e 65 anos',
    19: 'entre 66 e 70 anos',
    20: 'mais que 70 anos',
}

df['TP_FAIXA_ETARIA'] = (df['TP_FAIXA_ETARIA'].fillna(0).astype(int))

df['TP_FAIXA_ETARIA'] = df['TP_FAIXA_ETARIA'].map(dicio_faixaetaria)

print("Coluna TP_ETP_FAIXA_ETARIA")
print(df['TP_FAIXA_ETARIA'].head(5))
print("Valores únicos da coluna 'TP_FAIXA_ETARIA': \n", df['TP_FAIXA_ETARIA'].unique())


Coluna TP_ETP_FAIXA_ETARIA
0               20 anos
1    entre 31 e 35 anos
2    entre 36 e 40 anos
3               18 anos
4               17 anos
Name: TP_FAIXA_ETARIA, dtype: object
Valores únicos da coluna 'TP_FAIXA_ETARIA': 
 ['20 anos' 'entre 31 e 35 anos' 'entre 36 e 40 anos' '18 anos' '17 anos'
 '25 anos' '24 anos' '19 anos' '23 anos' 'entre 26 e 30 anos' '22 anos'
 'Menos que 17 anos' '21 anos' 'entre 41 e 45 anos' 'entre 51 e 55 anos'
 'entre 66 e 70 anos' 'entre 46 e 50 anos' 'mais que 70 anos'
 'entre 56 e 60 anos' 'entre 61 e 65 anos']


O nome da tabela então passará a ser idade

In [122]:
df.rename(columns={'TP_FAIXA_ETARIA': 'idade'}, inplace=True)
print("Coluna idade")
print(df['idade'].head(5))

Coluna idade
0               20 anos
1    entre 31 e 35 anos
2    entre 36 e 40 anos
3               18 anos
4               17 anos
Name: idade, dtype: object


## **Transformação da coluna TP_ANO_CONCLUIU**

In [123]:
print("Valores únicos da coluna 'TP_ANO_CONCLUIU': \n", df['TP_ANO_CONCLUIU'].unique())
print('Coluna TP_ANO_CONCLUIU')
print(df['TP_ANO_CONCLUIU'].head(20))

Valores únicos da coluna 'TP_ANO_CONCLUIU': 
 [ 3 11 15  0  9  6  1  2  4  8  5 10 14  7 13 12 16 17]
Coluna TP_ANO_CONCLUIU
0      3
1     11
2     15
3      0
4      0
5      9
6      6
7      1
8      0
9      0
10     2
11     0
12    11
13     3
14     0
15     1
16     4
17     6
18     0
19     9
Name: TP_ANO_CONCLUIU, dtype: int64


TP_ANO_CONCLUIU 16 possiveis valores em 2021, tem 17 possiveis valores em 2022 e 18 possiveis valores em 2023 sendo em todos: (0 = não informado) e o valor max(16, 17, 18 = antes de 2017), todos os outros valores sao uma sequencia do (ano atual - 1 até 2007)

## **Transformação da coluna Q001**

In [124]:
print("Valores únicos da coluna 'Q001': \n", df['Q001'].unique())
print('Coluna Q001')
print(df['Q001'].head(20))

Valores únicos da coluna 'Q001': 


 ['F' 'B' 'D' 'H' 'E' 'G' 'C' 'A']
Coluna Q001
0     F
1     B
2     B
3     B
4     D
5     H
6     B
7     E
8     B
9     D
10    E
11    D
12    E
13    B
14    G
15    D
16    E
17    E
18    B
19    E
Name: Q001, dtype: object


Q001 possui 8 possiveis valores( [A = Nunca Estudou], [B = Não completou o ensino fundamental 1], 
[C = Não completou o ensino fundamental 2], [D = Não completou o ensino médio], [E = Não completou a faculdade], [F = Não completou a pós-graduação],
[G = Completou a pós-graduação], [H = Não sei] ) e um nulo(nan = Não informado).

In [125]:


df['Q001'] = df['Q001'].fillna('Não informado')

df.loc[df['Q001'] == 'A', 'Q001'] = 'Nunca Estudou'
df.loc[df['Q001'] == 'B', 'Q001'] = 'Não completou o ensino fundamental 1'
df.loc[df['Q001'] == 'C', 'Q001'] = 'Não completou o ensino fundamental 2'
df.loc[df['Q001'] == 'D', 'Q001'] = 'Não completou o ensino médio'
df.loc[df['Q001'] == 'E', 'Q001'] = 'Não completou a faculdade'
df.loc[df['Q001'] == 'F', 'Q001'] = 'Não completou a pós-graduação'
df.loc[df['Q001'] == 'G', 'Q001'] = 'Completou a pós-graduação'
df.loc[df['Q001'] == 'H', 'Q001'] = 'Não sei'


print("Coluna Q001")
print(df['Q001'].head(5))
print("Valores únicos da coluna 'Q001': \n", df['Q001'].unique())

Coluna Q001
0           Não completou a pós-graduação
1    Não completou o ensino fundamental 1
2    Não completou o ensino fundamental 1
3    Não completou o ensino fundamental 1
4            Não completou o ensino médio
Name: Q001, dtype: object
Valores únicos da coluna 'Q001': 
 ['Não completou a pós-graduação' 'Não completou o ensino fundamental 1'
 'Não completou o ensino médio' 'Não sei' 'Não completou a faculdade'
 'Completou a pós-graduação' 'Não completou o ensino fundamental 2'
 'Nunca Estudou']


A coluna teve entao o nome trocado para escolaridade_pai

In [126]:
df.rename(columns={'Q001': 'escolaridade_pai'}, inplace=True)
print("Coluna escolaridade_pai")
print(df['escolaridade_pai'].head(5))

Coluna escolaridade_pai
0           Não completou a pós-graduação
1    Não completou o ensino fundamental 1
2    Não completou o ensino fundamental 1
3    Não completou o ensino fundamental 1
4            Não completou o ensino médio
Name: escolaridade_pai, dtype: object


## **Transformação e junção das colunas TP_PRESENCA_CN, TP_PRESENCA_MT**

Primeiro vou testar se os dados das duas colunas sao iguais

In [127]:
print("Valores únicos da coluna 'TP_PRESENCA_CN': \n", df['TP_PRESENCA_CN'].unique())
print("Valores únicos da coluna 'TP_PRESENCA_MT': \n", df['TP_PRESENCA_MT'].unique())

if(df['TP_PRESENCA_CN'].equals(df['TP_PRESENCA_MT'])):
    print('São iguais')

Valores únicos da coluna 'TP_PRESENCA_CN': 
 [0 1 2]
Valores únicos da coluna 'TP_PRESENCA_MT': 
 [0 1 2]
São iguais


Visto que todos os dados são iguais, vou utilizar TP_PRESENCA_MT para representar ambas as colunas e apagar a TP_PRESENCA_CN considerando que (0 = faltou à prova), (1 = presente na prova), (2 = eliminado na prova)

In [128]:
dicio_presencaMTCN = {
    0: 'faltou à prova',
    1: 'presente na prova',
    2: 'eliminado da prova',
}

df['TP_PRESENCA_MT'] = (df['TP_PRESENCA_MT'].fillna(0).astype(int))

df['TP_PRESENCA_MT'] = df['TP_PRESENCA_MT'].map(dicio_presencaMTCN)

print("Coluna TP_PRESENCA_MT")
print(df['TP_PRESENCA_MT'].head(5))
print("Valores únicos da coluna 'TP_PRESENCA_MT': \n", df['TP_PRESENCA_MT'].unique())


Coluna TP_PRESENCA_MT
0       faltou à prova
1    presente na prova
2       faltou à prova
3    presente na prova
4    presente na prova
Name: TP_PRESENCA_MT, dtype: object
Valores únicos da coluna 'TP_PRESENCA_MT': 
 ['faltou à prova' 'presente na prova' 'eliminado da prova']


In [129]:
df = df.drop('TP_PRESENCA_CN', axis=1)

In [130]:
if df.get('TP_PRESENCA_CN') is not None:
    print("A coluna existe")
else:
    print("A coluna não existe")


A coluna não existe


Agora vou apenas renomear 'TP_PRESENCA_MT' como 'presenca_2d'

In [131]:
df.rename(columns={'TP_PRESENCA_MT': 'presenca_2d'}, inplace=True)
print("Coluna presenca_2d")
print(df['presenca_2d'].head(5))

Coluna presenca_2d
0       faltou à prova
1    presente na prova
2       faltou à prova
3    presente na prova
4    presente na prova
Name: presenca_2d, dtype: object


## **Transformação e junção das colunas TP_PRESENCA_LC, TP_PRESENCA_CH**

O processo é idêntico à transformação e junção de TP_PRESENCA_CN e TP_PRESENCA_MT exceto pelo nome das variaveis

In [132]:
print("Valores únicos da coluna 'TP_PRESENCA_LC': \n", df['TP_PRESENCA_LC'].unique())
print('Coluna TP_PRESENCA_LC')
print("Valores únicos da coluna 'TP_PRESENCA_CH': \n", df['TP_PRESENCA_CH'].unique())
print('Coluna TP_PRESENCA_CH')

if(df['TP_PRESENCA_LC'].equals(df['TP_PRESENCA_CH'])):
    print('São iguais')

Valores únicos da coluna 'TP_PRESENCA_LC': 
 [1 0 2]
Coluna TP_PRESENCA_LC
Valores únicos da coluna 'TP_PRESENCA_CH': 
 [1 0 2]
Coluna TP_PRESENCA_CH
São iguais


In [133]:
dicio_presencaLCCH = {
    0: 'faltou à prova',
    1: 'presente na prova',
    2: 'eliminado da prova',
}

df['TP_PRESENCA_LC'] = (df['TP_PRESENCA_LC'].fillna(0).astype(int))

df['TP_PRESENCA_LC'] = df['TP_PRESENCA_LC'].map(dicio_presencaLCCH)

print("Coluna TP_PRESENCA_LC")
print(df['TP_PRESENCA_LC'].head(5))
print("Valores únicos da coluna 'TP_PRESENCA_LC': \n", df['TP_PRESENCA_LC'].unique())


Coluna TP_PRESENCA_LC
0    presente na prova
1    presente na prova
2       faltou à prova
3    presente na prova
4    presente na prova
Name: TP_PRESENCA_LC, dtype: object
Valores únicos da coluna 'TP_PRESENCA_LC': 
 ['presente na prova' 'faltou à prova' 'eliminado da prova']


In [134]:
df = df.drop('TP_PRESENCA_CH', axis=1)

In [135]:
if df.get('TP_PRESENCA_CH') is not None:
    print("A coluna existe")
else:
    print("A coluna não existe")


A coluna não existe


In [136]:
df.rename(columns={'TP_PRESENCA_LC': 'presenca_1d'}, inplace=True)
print("Coluna presenca_1d")
print(df['presenca_1d'].head(5))

Coluna presenca_1d
0    presente na prova
1    presente na prova
2       faltou à prova
3    presente na prova
4    presente na prova
Name: presenca_1d, dtype: object
